In [66]:
import pandas as pd
import matplotlib.pyplot as plt 
#import seaborn as sns
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss
from scipy import stats
from datetime import timedelta
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [67]:
def split_dataframe_by_sliding_window(df, time_column='Time', window_size='6H', step='15T'):
        df[time_column] = pd.to_datetime(df[time_column])
        
        df = df.sort_values(by=time_column)
        
        window_size_timedelta = pd.to_timedelta(window_size)
        step_timedelta = pd.to_timedelta(step)
        start_time = df[time_column].min()
        end_time = df[time_column].max()
        
        result = []
        
        while start_time <= end_time:
            window_end_time = start_time + window_size_timedelta
            window_df = df[(df[time_column] >= start_time) & (df[time_column] < window_end_time)]
            
            if not window_df.empty:
                result.append(window_df)
            
            start_time += step_timedelta
        
        return result


In [91]:
class DrillingLogs:

    # TODO: sliding window (6 hours) with step size of 15 minutes (1st July)
    # TODO: method that takes date and returns all outliers (1st July)
    
    IMPORTANT_COLUMNS = [
        'Time', 'Flow In', 'Bit RPM',
        'Total Depth', 'Top Drive Torque (ft-lbs)',
        'ROP Depth/Hour',  'Block Position',
        'Weight on Bit', 'Depth Hole TVD','Pit Volume Active',
        'Return Flow', 'Pit G/L Active'
    ]
    
    NUMERICAL_FEATURES= {
        'Block Position': (0, 1000),  # Feet or meters, depending on the rig setup, always positive
        'Weight on Bit': (0, 100),  # Tons, complimentory to hookload
        'Hookload': (50, 600),  # Tons
        'ROP Depth/Hour': (0, 200),  # Feet per hour, derived and computed automatically
        'MWD Gamma (API)': (0, 150),  # API units
        'Top Drive RPM': (0, 300),  # RPM
        'Top Drive Torque (ft-lbs)': (0, 60000),  # Foot-pounds
        'Flow In': (0, 1200),  # Gallons per minute
        'Pump Pressure': (500, 5000),  # PSI
        'SPM Total': (0, 250),  # Strokes per minute
        'Pit Volume Active': (0, 1000),  # Barrels
        'Pit G/L Active': (0, 10),  # Gas/Liquid ratio
        'Gas Total - units': (0, 100),  # Units of gas detection
        'Trip Volume Active': (0, 1000),  # Barrels
        'Trip G/L': (0, 10),  # Gas/Liquid ratio
        'Return Flow': (0, 1000),  # Gallons per minute
        'RES PS 2MHZ 18IN': (0, 2000),  # Ohm-meters
        'RES PS 400KHZ 18IN': (0, 2000),  # Ohm-meters
        'MWD Inclination': (0, 90),  # Degrees
        'MWD Azimuth': (0, 360),  # Degrees
        'H2S 01': (0, 10),  # Parts per million (ppm)
        'RSS Azimuth': (0, 360),  # Degrees
        'Total Depth': (0, 30000),  # Feet
        'Bit Diameter': (4, 20),  # Inches
        'Bit RPM': (0, 200),  # RPM
        'Depth Hole TVD': (0, 30000),  # Feet
        'Differential Pressure': (0, 5000),  # PSI
        'Downhole Torque': (0, 60000),  # Foot-pounds
        'MUD TEMP': (0, 200)  # Degrees Fahrenheit
    }

    CATEGORICAL_FEATURES = {
        'Slips Set': (0, 1),  # Binary, 0 or 1
        'On Bottom': (0, 1),  # Binary, 0 or 1
        'RigMode': (0, 10),  # Categorical, specific to rig operations
        'ROCKIT - On/Off': (0, 1),  # Binary, 0 or 1
        'RigEventCode': (0, 9999),  # Categorical, specific to rig events
        'Drill Mode': (0, 5)  # Categorical, specific to drilling operations
    }



    def __init__(self, file_name):
        if '.csv' in file_name:
            self.df = pd.read_csv(file_name)
            self.df['Time'] = pd.to_datetime(self.df['Time'])
        
        else:
            self.df = pd.read_excel(file_name)
            self.df['Time'] = pd.to_datetime(self.df['Time'])
        
        self.df.replace(-999.25, np.nan, inplace = True)
        self.columns = self.df.columns

    def split_dataframe_by_sliding_window(self, time_column='Time', window_size='6H', step='15T'):
        self.df[time_column] = pd.to_datetime(self.df[time_column])
        
        df = self.df.sort_values(by=time_column)
        
        window_size_timedelta = pd.to_timedelta(window_size)
        step_timedelta = pd.to_timedelta(step)
        start_time = df[time_column].min()
        end_time = df[time_column].max()
        
        result = []
        
        while start_time <= end_time:
            window_end_time = start_time + window_size_timedelta
            window_df = df[(df[time_column] >= start_time) & (df[time_column] < window_end_time)]
            
            if not window_df.empty:
                result.append(window_df)
            
            start_time += step_timedelta
        
        return result
    

    
    def get_correlations(self, threshold=0.5, subset = 'important'):
        if subset == 'important':
            corr_matrix = self.df[self.IMPORTANT_COLUMNS[1:]].corr()
        elif subset == 'all':
            corr_matrix = self.df[[self.df.columns[1:]]].corr()

        # Dictionary to store pairs of correlated features
        correlated_features = {}

        # Iterate through the correlation matrix
        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                feature1 = corr_matrix.columns[i]
                feature2 = corr_matrix.columns[j]
                correlation = corr_matrix.iloc[i, j]
                
                # Check if the correlation is above the threshold
                if abs(correlation) >= threshold:
                    correlated_features[(feature1, feature2)] = correlation

        return correlated_features
    
    
    def get_outliers(self, date, subset='important'):
        
        result = []
        
        if subset == 'important':
            for col in self.IMPORTANT_COLUMNS[1:]:
                temp_df = self.df[['Time', col]]
                filtered_df = temp_df[temp_df['Time'].dt.date == pd.to_datetime(date).date()]
                filtered_df = filtered_df.drop_duplicates(subset='Time')
                series = filtered_df.dropna()
                series_windows = split_dataframe_by_sliding_window(series)
                for s in series_windows:
                    # IQR method
                    Q1 = s[col].quantile(0.25)
                    Q3 = s[col].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR
                    iqr_outliers = s[(s[col] < lower_bound) | (s[col] > upper_bound)]
                  
                    # Z-score method
                    z_scores = stats.zscore(s[col])
                    z_outliers = s[(np.abs(z_scores) > 3)]
            

                    # range based method
                    range_based_outliers = s[(s[col] < self.NUMERICAL_FEATURES[col][0]) | (s[col] > self.NUMERICAL_FEATURES[col][1])]
                    
                    final_outliers = pd.merge(iqr_outliers, range_based_outliers, on='Time', how='inner')
                    final_outliers.rename(columns = {col + '_x' : col}, inplace=True)
                    final_outliers = final_outliers[['Time', col]]

                    final_outliers = pd.merge(final_outliers, z_outliers, on='Time', how='inner')
                    final_outliers.rename(columns = {col + '_x' : col}, inplace=True)
                    final_outliers = final_outliers[['Time', col]]
                    
                    for index, row in final_outliers.iterrows():
                        result.append(f"At {row['Time']} {col} is out of range with value {row[col]}")

        elif subset == 'all':
            for col in self.NUMERICAL_FEATURES:
                temp_df = self.df[['Time', col]]
                filtered_df = temp_df[temp_df['Time'].dt.date == pd.to_datetime(date).date()]
                filtered_df = filtered_df.drop_duplicates(subset='Time')
                series = filtered_df.dropna()
                series_windows = self.split_dataframe_by_sliding_window(series)
                for s in series_windows:
                    # IQR method
                    Q1 = s[col].quantile(0.25)
                    Q3 = s[col].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR
                    iqr_outliers = s[(s[col] < lower_bound) | (s[col] > upper_bound)]
                  
                    # Z-score method
                    z_scores = stats.zscore(s[col])
                    z_outliers = s[(np.abs(z_scores) > 3)]
            

                    # range based method
                    range_based_outliers = s[(s[col] < self.NUMERICAL_FEATURES[col][0]) | (s[col] > self.NUMERICAL_FEATURES[col][1])]
                    
                    final_outliers = pd.merge(iqr_outliers, range_based_outliers, on='Time', how='inner')
                    final_outliers.rename(columns = {col + '_x' : col}, inplace=True)
                    final_outliers = final_outliers[['Time', col]]

                    final_outliers = pd.merge(final_outliers, z_outliers, on='Time', how='inner')
                    final_outliers.rename(columns = {col + '_x' : col}, inplace=True)
                    final_outliers = final_outliers[['Time', col]]
                    
                    for index, row in final_outliers.iterrows():
                        result.append(f"At {row['Time']} {col} is out of range with value {row[col]} ")
                    
        result_df = pd.DataFrame({'alert' : result})
        result_df.drop_duplicates(inplace=True)

        return list(result_df['alert'])
        
    
    def describe(self, subset = 'all'):
        if subset == 'all':
            return self.df.describe()
        elif subset == 'important':
            return self.df[self.IMPORTANT_COLUMNS].describe()
        
    def get_report(self, date, subset='important'):
        correlations = self.get_correlations(subset=subset)
        outliers = self.get_outliers(subset=subset, date=date)
        stat_df = self.describe(subset=subset)

        list_of_correlations = []
        for k,v in correlations.items():
            list_of_correlations.append(f'Correlation between {k[0]} and {k[1]} is equal to {v}')
        
        stat_df = self.describe(subset=subset)
        

        lines_to_write = []
       
        lines_to_write.append(f'Outliers on date {date} with window_size = 6 hrs and step = 15 min: \n')
            
        lines_to_write.append('\n'.join(outliers))

        lines_to_write.append('\n')

        lines_to_write.append(f'Correlations between {subset} features:\n')
        lines_to_write.append('\n'.join(list_of_correlations))
        lines_to_write.append('\n')

        for col in stat_df.columns[1:]:
            lines_to_write.append(f'Statistics for {col}:\n')
            lines_to_write.append(f'Mean value = {stat_df[col][1]}\n')
            lines_to_write.append(f'Minimum value = {stat_df[col][2]}\n')
            lines_to_write.append(f'25th percentile = {stat_df[col][3]}\n')
            lines_to_write.append(f'Median = {stat_df[col][4]}\n')
            lines_to_write.append(f'75th percentile = {stat_df[col][5]}\n')
            lines_to_write.append(f'Maximum value = {stat_df[col][6]}\n')
            lines_to_write.append(f'Standard deviation = {stat_df[col][7]}\n')
            lines_to_write.append('\n')
        print('Report is ready in the same folder!')
        
        return ''.join(lines_to_write)
        
 

  
     

In [69]:
  
    def plot_time_series(self, column_name):
        df = self.df[['Time', column_name]]
        df.dropna(subset=column_name)
        plt.figure(figsize=(10, 6))
        plt.plot(df['Time'], df[column_name], label=f'Time Series for {column_name}')
        plt.xlabel('Date')
        plt.ylabel(column_name)
        plt.title(f'Time Series Plot for {column_name}')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_correlation_heatmap(self, subset):
        if subset == 'important':
            plt.figure(figsize=(10, 8))
            sns.heatmap(self.df[[self.IMPORTANT_COLUMNS]].corr(), annot=False, cmap='coolwarm', vmin=-1, vmax=1)
            plt.title('Correlation Heatmap')
            plt.show()

        elif subset == 'all':    
            plt.figure(figsize=(10, 8))
            sns.heatmap(self.df.corr(), annot=False, cmap='coolwarm', vmin=-1, vmax=1)
            plt.title('Correlation Heatmap')
            plt.show()

In [92]:
ddr = DrillingLogs('10s_intervals.csv')

In [93]:
print(ddr.get_report(subset='important', date='2020-10-25'))

Report is ready in the same folder!
Outliers on date 2020-10-25 with window_size = 6 hrs and step = 15 min: 
At 2020-10-25 15:31:00 Flow In is out of range with value 1280.24212
At 2020-10-25 18:11:00 Block Position is out of range with value -0.18317
At 2020-10-25 18:12:00 Block Position is out of range with value -0.1832
At 2020-10-25 18:13:00 Block Position is out of range with value -0.18321
At 2020-10-25 18:14:00 Block Position is out of range with value -0.1832
At 2020-10-25 19:00:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:01:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:02:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:03:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:04:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:05:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:06:00 Pit G/L Active is out of range with value -183.645
At 2020-10

In [72]:
l = ddr.describe(subset='important')

In [73]:
l

,Time,Flow In,Bit RPM,Total Depth,Top Drive Torque (ft-lbs),ROP Depth/Hour,Block Position,Weight on Bit,Depth Hole TVD,Pit Volume Active,Return Flow,Pit G/L Active
count,608676,607502.000000,524965.000000,596875.000000,596861.000000,596864.000000,596850.000000,596864.000000,597426.000000,607502.000000,596861.000000,319934.000000
mean,2020-11-30 11:29:37.048971264,378.243915,98.353336,7084.081454,1612.587569,11.258759,44.968921,72.997994,7085.032327,457.868218,30.489147,-7.933645
min,2020-10-25 12:38:00,0.000000,0.000000,0.000020,0.000000,0.000000,-1511.294800,-15599.576000,0.000020,0.000000,0.000000,-554.780150
25%,2020-11-12 15:37:45,0.000000,0.251440,5123.895000,0.000000,0.000000,13.453960,2.177595,5123.895000,426.309183,2.226410,-27.962595
50%,2020-11-30 12:35:00,53.988380,16.458000,7389.890000,0.000000,0.000000,43.595820,46.957640,7389.890000,490.567140,31.993450,-4.891470
75%,2020-12-18 08:18:00,563.394650,145.513050,10722.805000,2169.388200,0.000000,79.778398,172.120663,10737.555500,518.068240,57.506680,13.560185
max,2021-01-05 11:01:00,132667.200000,377404.160000,735423.000000,31021.443000,147628.900000,116.247670,837.661990,735423.000000,679.944760,100.049600,247.985000
std,NaN,3538.181459,1614.408787,3417.365815,3171.310006,445.271471,66.410902,115.733009,3416.793139,93.369311,27.651717,65.671203


In [74]:
for col in l.columns[1:]:
    print(f'Statistics for {col}:')
    print(f'Mean value = {l[col][1]:.3f}')
    print(f'Minimum value = {l[col][2]:.3f}')
    print(f'25th percentile = {l[col][3]:.3f}')
    print(f'Median = {l[col][4]:.3f} ')
    print(f'75th percentile = {l[col][5]:.3f}')
    print(f'Maximum value = {l[col][6]:.3f}')
    print(f'Standard deviation = {l[col][7]:.3f}')
    print('\n')

Statistics for Flow In:
Mean value = 378.244
Minimum value = 0.000
25th percentile = 0.000
Median = 53.988 
75th percentile = 563.395
Maximum value = 132667.200
Standard deviation = 3538.181


Statistics for Bit RPM:
Mean value = 98.353
Minimum value = 0.000
25th percentile = 0.251
Median = 16.458 
75th percentile = 145.513
Maximum value = 377404.160
Standard deviation = 1614.409


Statistics for Total Depth:
Mean value = 7084.081
Minimum value = 0.000
25th percentile = 5123.895
Median = 7389.890 
75th percentile = 10722.805
Maximum value = 735423.000
Standard deviation = 3417.366


Statistics for Top Drive Torque (ft-lbs):
Mean value = 1612.588
Minimum value = 0.000
25th percentile = 0.000
Median = 0.000 
75th percentile = 2169.388
Maximum value = 31021.443
Standard deviation = 3171.310


Statistics for ROP Depth/Hour:
Mean value = 11.259
Minimum value = 0.000
25th percentile = 0.000
Median = 0.000 
75th percentile = 0.000
Maximum value = 147628.900
Standard deviation = 445.271


Stat

In [77]:
with open('output.txt','r') as file:
    content_list = file.readlines()
for line in content_list:
    print(line.strip())

Outliers on date 2020-10-25 with window_size = 6 hrs and step = 15 min:
At 2020-10-25 15:31:00 Flow In is out of range with value 1280.24212
At 2020-10-25 18:11:00 Block Position is out of range with value -0.18317
At 2020-10-25 18:12:00 Block Position is out of range with value -0.1832
At 2020-10-25 18:13:00 Block Position is out of range with value -0.18321
At 2020-10-25 18:14:00 Block Position is out of range with value -0.1832
At 2020-10-25 19:00:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:01:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:02:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:03:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:04:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:05:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:06:00 Pit G/L Active is out of range with value -183.645
At 2020-10-25 19:07:00 Pit G/L Active is out of

In [ ]:
with open('output.txt', 'r') as file:
    content_list = file.readlines()

# Now content_list contains each line of the file as an element in the list
for line in content_list:
    print(line.strip())  # Print each line (strip() removes newline characters)

# Optionally, you can also process specific parts of the content_list
# For example, if you want to retrieve statistics for a particular column:
# Assuming 'Statistics for col1:' is one of the lines
statistics_for_col1 = []
for index, line in enumerate(content_list):
    if line.startswith('Statistics for col1:'):
        # Collect the lines following this header until a blank line or end of file
        index += 1
        while index < len(content_list) and content_list[index].strip():
            statistics_for_col1.append(content_list[index].strip())
            index += 1
        break

# Print or process the collected statistics for col1
print("Statistics for col1:")
for stat_line in statistics_for_col1:
    print(stat_line)